<a href="https://colab.research.google.com/github/BrunoShinniti/transformer-mecanismo-de-atencao/blob/main/Mecanismo_de_Aten%C3%A7%C3%A3o_(Transformer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## com PyTorch

In [ ]:
!pip install -q -U torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 118.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn

In [ ]:
class Transformer(nn.Module):

    def __init__(self, vocab_size, embedding_dim, n_heads, n_layers, dropout):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.drouptout = dropout

        # Define a camada de embedding que transformará a sequência de entrada em uma sequência de vetores de dimensão fixa
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Define o mecanismo de auto-atenção multi-head
        self.attention = nn.MultiheadAttention(embedding_dim, n_heads, dropout = dropout)

        # Define a rede neural feed-forward qe será usada para gerar a sequência de saída a partir da sequência de entrada
        self.feed_forward = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )

        # Define a camada de saída final que transformará a sequência de saída na forma de saída desejada
        self.out = nn.Linear(embedding_dim, vocab_size)


    def forward(self, x):

        # Aplica a camada de embedding à sequência de entrada
        x = self.embedding(x)

        # Aplica o mecanismo multi-headed self-attetion
        x = self.attention(x)

        # Aplica o método feed-forward
        x = self.feed_forward(x)

        # Aplica camada final
        x = self.out(x)

        return x


modelo = Transformer(vocab_size = 1000,
                     embedding_dim = 32,
                     n_heads = 4,
                     n_layers = 2,
                     dropout = 0.5)

modelo.modules

<bound method Module.modules of Transformer(
  (embedding): Embedding(1000, 32)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
  )
  (feed_forward): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (out): Linear(in_features=32, out_features=1000, bias=True)
)>

In [ ]:
modelo.attention

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
)

## Sem Framework

Construção de modelo capaz de prever sequências de comprimento igual a 10 tokens.

* 1 - Camada de Embedding: Transforma as palavras em vetores numéricos de tamanho fixo.

* 2 - Mecanismo de Atenção: Permite que o modelo foque em diferentes partes da entrada.

* 3 - Camadas Encoder e Decoder: Processam os dados sequencialmente

* 4 - Camada Linear e Softmax: Para predições finais

#### Objetivo:
Implementar o item 2, mas para deixar o exemplo funcional, será implementado os itens 1 e 4.

### Hiperparâmetros Iniciais

In [1]:
import numpy as np

In [13]:
# Dimensão do modelo
dim_model = 64

# Comprimento da sequência
seq_lenght = 10

# Tamanho do vocabulário
vocab_size = 100

### Camada Embedding

A função embedding é utulizada para converter entradas sequenciais em vetores densos de tamanho fixo. Esses vetores são conhecimdos como embeddings e são uma parte fundamental, em especial dos modelos PLN.<br><br>
Esses embeddings são fundamentais para modelos de aprendizado profundo em PLN, pois fornecem uma representação rica e densa de palavras ou tokens, capturando informações contextuais e semânticas que são essenciais para tarefas como tradução automática, classificação de texto, entre outras.

In [3]:
def embedding(input, vocab_size, dim_model):

    # Cria uma matriz de embedding onde cada linha representa um token de vocabulário
    # A matriz é inicializada com valores aleatórios normalmente distrbuídos
    embed = np.random.randn(vocab_size, dim_model)

    return np.array([embed[i] for i in input])

### Softmax
A função softmax é uma função de ativação amplamente utilizada em redes neurais, especialmente em cenários de classificação, onde é importante transformar valores brutos de saída (logits) em probabilidades que somam 1. Abaixo, está o código da função softmax com comentários em cada linha explicando seu funcionamento:

In [9]:
def softmax(x):

    # Calcula o exponencial de cada elemtno do input, ajustado pelo máximo valor no input para evitar overflow numérico
    e_x = np.exp(x - np.max(x))

    # Divide cada exponencial pelo somatório dos exponenciais ao longo do último eixo (axis = -1)
    # O Reshape(-1, 1) garante que a divisão seja realizada corretamente em um contexto multidimensional
    return e_x / e_x.sum(axis=-1).reshape(-1, 1)

### Scale dot product

A função scaled_dot_product_attention() é um componente do mecanismo de atenção em modelos Transformer. Ela calcula a atenção entre conjuntos de queries (Q), keys (K) e values (V).

Essencialmente, essa função permite que o modelo dê importância diferenciada a diferentes partes da entrada, um aspecto chave que torna os modelos Transformer particularmente eficazes para tarefas de PLN e outras tarefas sequenciais.

In [6]:
# Define a função para calcular a atenção escalada por produto escalar
def scaled_dot_product_attention(Q, K, V):

    # Calcula o produto escalar entre Q e a transposta de K
    matmul_qk = np.dot(Q, K.T)

    # Obtém a dimensão dos vetores de chave
    depth = K.shape[-1]

    # Escala os logits dividindo-os pela raiz quadrada de profundidade
    logits = matmul_qk / np.sqrt(depth)

    # Aplica a função softmax para obter os pesos de atenção
    attention_weights = softmax(logits)

    # Multiplica os pesos de atenção pelos valores V para obter a saída final
    output = np.dot(attention_weights, V)

    return output

### Saída do Modelo com operação Linear e Softmax

A função linear_and_softmax( ) é uma combinação de uma camada linear seguida por uma função softmax, comumente usada em modelos de aprendizado profundo, especialmente em tarefas de classificação.

In [7]:
# Define a função que aplica uma transformação linear seguida de softmax

def linear_and_softmax(input):

    # Inicializa uma matriz de pesos com valores aleatórios normamente distribuídos
    # Esta matriz conecta cada dimensão do modelo (dim_model)  a cada palavra do vocabulário (vocab_size)
    weights = np.random.randn(dim_model, vocab_size)

    # Realiza a operação linear (produto escalar) entre a entrada e a matriz de pesos
    # O resultado, logits, é um vetor que representa a entrada transformada em um espaço de maior dimensão
    logits = np.dot(input, weights)

    # Aplica a função softmax aos logits
    # Isso transforma os logits em um vetor de probabilidades, onde cada elemento soma 1

    return softmax(logits)

### Modelo Final

In [11]:
def transformer_model(input):

    # Embedding
    embedded_input = embedding(input, vocab_size, dim_model)

    # Mecanismo de atenção
    attention_output = scaled_dot_product_attention(embedded_input, embedded_input, embedded_input)

    # Camada linear e softmax
    output_probabilities = linear_and_softmax(attention_output)

    # Escolhendo os índices com maior probabilidade
    output_indices = np.argmax(output_probabilities, axis=-1)

    return output_indices

## Usando modelo para previsão

In [17]:
input_sequence = np.random.randint(0, vocab_size, seq_lenght)
input_sequence

array([37, 97, 12,  5, 54, 59, 68, 20, 33, 34])

In [18]:
output = transformer_model(input_sequence)

In [19]:
output

array([ 0, 31, 87, 64, 31, 79, 25, 60, 12, 32])